### Assign hydropower plant to their nearest reservoir

Author : Jignesh Shah, Jing Hu, Oreane Edlenbosch and Michelle T.H van Vliet

#### Requirements

Following packages are required

1. xarray
2. rioxarray
3. numpy
4. geopands
5. shapely
6. geopy
7. pandas



#### Files required to be downloaded

The GloHydroRes dataset is a compilation of open-source hydropower plant and reservoir data. It combines information about hydropower plants (such as location, capacity, water head, and type) with reservoir characteristics (including dam and reservoir location, dam height, reservoir depth, area, and volume). Below mentioned datasets are used in GloHydroRes dataset.

1. Global Power Plant Database from World Resource Institute (WRI) [https://datasets.wri.org/dataset/globalpowerplantdatabase]
2. Hydropower Infrastruture - LAkes, Reservoirs, and RIvers (HILARRI), Version 2 [https://hydrosource.ornl.gov/dataset/hilarri-v2]
3. Existing Hydropower Assets (EHA) Plant Database, 2022 [https://hydrosource.ornl.gov/dataset/EHA2022]
4. JRC Hydro-power plants database [https://github.com/energy-modelling-toolkit/hydro-power-database/]
5. RePP Africa - Renewable Power Plant database for Africa [https://www.nature.com/articles/s41597-022-01922-1]
6. Global Reservoir and Dam (GranD) database [https://depts.washington.edu/saswe/grand/GRanD_Technical_Documentation_v1_1.pdf]
6. Georeferenced global Dams and Reservoir (GeoDAR) [https://essd.copernicus.org/articles/14/1869/2022/]
7. Global Dam Tracker (GDAT) [https://www.nature.com/articles/s41597-023-02008-2]
8. HydroLAKES [https://www.hydrosheds.org/products/hydrolakes]
9. HydroSHEDs 15 arc-second DEM is used to determine the elevation of hydropower plants and dams. It can be download from https://www.hydrosheds.org/hydrosheds-core-downloads.


In [2]:
# Importing the required libraries
import pandas as pd
from rioxarray import open_rasterio
from geopandas import read_file
from xarray import open_dataset
from shapely.ops import nearest_points # For finding the nearest point
from geopy.distance import geodesic # For calculating the distance between two points (coordinates)
import geopandas as gpd
import xarray as xr
from shapely.geometry import Point
import numpy as np

In [3]:
def load_input_data(folder_name, file_type, file_name):
    """ Load the input data

    : param folder_type: str: folder name of the input data
    : param file_type: str: type of the input data
    : param file_name: str: file name of the input data
    : return data: pd.DataFrame, xr.Dataset, gpd.GeoDataFrame: input data
    
    """
    # get path to the input data

    input_path = f"{folder_name}/{file_name}"

    if file_type == "tif":
        data = open_rasterio(input_path)
        data =  data.isel(band=0)

    elif file_type == "shp":
        data = read_file(input_path)
    
    elif file_type == "nc":
        data = open_dataset(input_path)
    
    elif file_type == "csv":
        data = pd.read_csv(input_path)

    else:
        raise ValueError(f"File type {file_type} is not supported")

    return data

For hydropower following datasets were used : 
1) WRI 
2) EHA 
3) RePP
4) JRC

For Reservoir following datasets were used:  
1) GranD
2) GeoDAR 
3) HydroLakes

Unique id used in each dataset
1) WRI - gppd_idnr
2) RePP - ID
3) JRC - id
4) EHA - eha_ptid
5) GranD - grand_id
6) GeoDAR - id_v11
7) HydroLakes - Hylak_id


Columns selected from each hydropower dataset
1) WRI - country_long, name, capacity_mw, latitude, longitude, gppd_idnr, commissioning_year
2) JRC - id, name, installed_capacity_MW, type, lat, lon, dam_height_m, volume_Mm3, country
3) EHA and HILARRI - EHA_PtID, MW, Head_ft, OpYear, grand_id, pt_name, pt_lat, pt_lon
4) RePP - HPPD_ID, lon, lat, country, HPP_name, type, main_river, g_hgt_m, first_o_start, g_cap_mw

Columns selected from each reservoir dataset
1) GranD - GRAND_ID, RES_NAME, DAM_NAME, RIVER, DAM_HGT_M, AREA_SKM (Representative surface area of reservoir in km2), CAP_MCM (Representative maximum storage capacity of reservoir in million cubic meters), DEPTH_M, 
2) GeoDAR - id_v11
3) HydroLakes - Hylak_id, Lake_name, Lake_area, Vol_total and Depth_avg
3) GDAT - Area_Con (Surface area of the reservoir in square kilometers, consolidated from polygon, official, max, min), Volume_Con (Maximum storage capacity of the reservoir in million cubic meters, consolidated from official, max, min), Avg_Depth, Dam_Name, Reservoir, River, Height



In [4]:
#Open WRI data.
WRI_power_plant = load_input_data("/home/shah0012/GloHydroRes/Input_data/world_resource_institute_data/", "csv", "global_power_plant_database.csv")
WRI_hydro = WRI_power_plant[WRI_power_plant.primary_fuel == "Hydro"]
WRI_hydro = gpd.GeoDataFrame(WRI_hydro, geometry=gpd.points_from_xy(WRI_hydro.longitude, WRI_hydro.latitude), crs="EPSG:4326")
WRI_hydro = WRI_hydro[["country_long", "name", "capacity_mw", "latitude", "longitude", "gppd_idnr", "commissioning_year"]]
WRI_hydro.rename(columns={"country_long": "country", "latitude": "plant_lat", "longitude": "plant_lon", "gppd_idnr": "plant_source_id", "commissioning_year" : "year"}, inplace=True)
WRI_hydro["plant_source"] = "WRI"

/home/shah0012/.conda/envs/python_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [5]:
# Open JRC hydropower data
JRC_hydropower_data = pd.read_csv("https://raw.githubusercontent.com/energy-modelling-toolkit/hydro-power-database/master/data/jrc-hydro-power-plant-database.csv", sep=",")
geometry = [Point(xy) for xy in zip(JRC_hydropower_data.lon, JRC_hydropower_data.lat)]
crs = {'init': 'epsg:4326'}
JRC_hydropower_data = gpd.GeoDataFrame(JRC_hydropower_data, crs=crs, geometry=geometry)


# Convert volume from Mm3 to km3
JRC_hydropower_data["res_vol_km3"] = JRC_hydropower_data.volume_Mm3*0.001


# Country code to country name. JRC does not provide country name instead it provides country code
country_code_name_dict = {'CH' : "Switzerland",
'FR' : "France",
'IT' : "Italy",
'AT' : "Austria",
'ES' : "Spain",
'PT' : "Portugal",
'NO' : "Norway",
'SE' : "Sweden",
'FI' : "Finland",
'SK' : "Slovakia",
'RO' : "Romania",
'BG' : "Bulgaria",
'GR' : "Greece",
'AL' : "Albania",
'ME' : "Montenegro",
'BA' : "Bosnia and Herzegovina",
'HR' : "Croatia",
'SI' : "Slovenia",
'HU' : "Hungary",
'RS' : "Serbia",
'MK' : "North Macedonia",
'XK' : "Kosovo",
'PL' : "Poland",
'UA' : "Ukraine",
'BY' : "Belarus",
'DE' : "Germany",
'NL' : "Netherlands",
'BE' : "Belgium",
'LU' : "Luxembourg",
'IE' : "Ireland",
'UK' : "United Kingdom",
'DK' : "Denmark",
'EE' : "Estonia",
'LV' : "Latvia",
'LT' : "Lithuania",
'CZ' : "Czech Republic"}


JRC_hydropower_data["country"] = JRC_hydropower_data.country_code.map(country_code_name_dict)

JRC_hydropower_data = JRC_hydropower_data[["id", "name", "installed_capacity_MW", "type", "lat", "lon", "dam_height_m", "res_vol_km3", "country"]]
JRC_hydropower_data.rename(columns={"id": "plant_source_id", "name": "name", "installed_capacity_MW": "capacity_mw", 
                                    "type" : "plant_type", "lat" : "plant_lat", "lon" : "plant_lon"}, inplace=True)

JRC_hydropower_data["plant_source"] = "JRC"
JRC_hydropower_data["res_attr_source"] = np.where(pd.isna(JRC_hydropower_data["res_vol_km3"]), np.nan, "JRC")
JRC_hydropower_data["dam_height_source"] = np.where(pd.isna(JRC_hydropower_data["dam_height_m"]), np.nan, "JRC")


def change_type(row):
    if row['plant_type'] == 'HDAM':
        return 'STO'
    elif row['plant_type'] =='HROR':
        return 'ROR'
    elif row['plant_type'] == "HPHS": 
        return "PS"
    
JRC_hydropower_data["plant_type"] = JRC_hydropower_data.apply(change_type, axis=1)
JRC_hydropower_data["plant_type_source"] = np.where(pd.isna(JRC_hydropower_data["plant_type"]), np.nan, "JRC")

/home/shah0012/.conda/envs/python_env/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
# US hydropower data
# EHA provide hydropower installed capacity and head data.
# HILARRI provides hydropower link to grand data
# Both EHA and HILARRI have eha_ptid which is common between them. Therefore we can link the data using eha_ptid

EHA_unit_level_hydropower_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/USA_ORNL_EHAHydroUnit_FY2023", "shp", "Unit_external2023.shp")
USA_HILARRI_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/HILARRI_v1_1_Shapefile", "shp", "HILARRI_v1_1_Public.shp")
EHA_unit_level_hydropower_data["head_m"] = EHA_unit_level_hydropower_data.Head_ft * 0.3048 # Convert head from ft to m
USA_HILARRI_data.grand_id =  np.where(USA_HILARRI_data.grand_id == "NA", np.nan, USA_HILARRI_data.grand_id)
USA_HILARRI_data.grand_id = USA_HILARRI_data.grand_id.astype(float)


# EHA provides data at unit level. We need to aggregate the data at plant level. Here sum the capacity, take the mean of the head. 
# Operation year is taken as the maximum of the operation year of the units
EHA_plant_level_hydropower_data = EHA_unit_level_hydropower_data.groupby(["EHA_PtID"]).MW.sum().reset_index()
EHA_plant_level_hydropower_data["head_m"] = EHA_unit_level_hydropower_data.groupby(["EHA_PtID"]).head_m.mean().reset_index()["head_m"]
EHA_plant_level_hydropower_data["year"] = EHA_unit_level_hydropower_data.groupby(["EHA_PtID"]).OpYear.max().reset_index()["OpYear"]
EHA_plant_level_hydropower_data["head_source"] = "ORNL" # ORNL is for Oak Ridge National Laboratory which is the source of both EHA and HILARRI data


eha_ptid_pt_name_dict = dict(zip(USA_HILARRI_data.eha_ptid, USA_HILARRI_data.pt_name)) # As EHA provides turbine name therefore plant name is taken from HILARRI data
eha_ptid_longitude_dict = dict(zip(USA_HILARRI_data.eha_ptid, USA_HILARRI_data.pt_lon)) # As EHA provides turbine longitude therefore plant longitude is taken from HILARRI data
eha_ptid_latitude_dict = dict(zip(USA_HILARRI_data.eha_ptid, USA_HILARRI_data.pt_lat)) # As EHA provides turbine latitude therefore plant latitude is taken from HILARRI data
eha_ptid_grand_id_dict = dict(zip(USA_HILARRI_data.eha_ptid, USA_HILARRI_data.grand_id)) # eha_ptid is linked to grand_id in HILARRI data


EHA_plant_level_hydropower_data["plant_name"] = EHA_plant_level_hydropower_data.EHA_PtID.map(eha_ptid_pt_name_dict)
EHA_plant_level_hydropower_data["plant_lat"] = EHA_plant_level_hydropower_data.EHA_PtID.map(eha_ptid_latitude_dict)
EHA_plant_level_hydropower_data["plant_lon"] = EHA_plant_level_hydropower_data.EHA_PtID.map(eha_ptid_longitude_dict)
EHA_plant_level_hydropower_data["res_dam_source_id"] = EHA_plant_level_hydropower_data.EHA_PtID.map(eha_ptid_grand_id_dict)
EHA_plant_level_hydropower_data["res_dam_source"] = np.where(EHA_plant_level_hydropower_data.res_dam_source_id.isna(), np.nan, "GranD")

EHA_plant_level_hydropower_data = EHA_plant_level_hydropower_data[~pd.isna(EHA_plant_level_hydropower_data.plant_name)] # Remove the plants which do not have name


# Convert lat and lon to float
EHA_plant_level_hydropower_data.plant_lat = np.where(EHA_plant_level_hydropower_data.plant_lat == "NA", np.nan, EHA_plant_level_hydropower_data.plant_lat)
EHA_plant_level_hydropower_data.plant_lon = np.where(EHA_plant_level_hydropower_data.plant_lon == "NA", np.nan, EHA_plant_level_hydropower_data.plant_lon)
EHA_plant_level_hydropower_data.plant_lat = EHA_plant_level_hydropower_data.plant_lat.astype(float)
EHA_plant_level_hydropower_data.plant_lon = EHA_plant_level_hydropower_data.plant_lon.astype(float)


EHA_plant_level_hydropower_data["country"] = "United States of America"


# Convert the data to geodataframe
geometry = [Point(xy) for xy in zip(EHA_plant_level_hydropower_data.plant_lon, EHA_plant_level_hydropower_data.plant_lat)]
crs = {'init': 'epsg:4326'}
EHA_plant_level_hydropower_data = gpd.GeoDataFrame(EHA_plant_level_hydropower_data, crs=crs, geometry=geometry)

EHA_plant_level_hydropower_data = EHA_plant_level_hydropower_data[['EHA_PtID', 'MW', 'head_m', 'head_source', 'plant_name', 'plant_lat', 'plant_lon', 'year', 'res_dam_source_id', 'res_dam_source']]

EHA_plant_level_hydropower_data.rename(columns={"MW": "capacity_mw", "EHA_PtID" : "plant_source_id", "plant_name" : "name"}, inplace=True)


EHA_plant_level_hydropower_data["plant_source"] = "EHA"

## As EHA data already contain grand id, we can directly get reservoir data from GranD data

# GranD dam data
grand_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/GRanD_Version_1_3", "shp", "GRanD_dams_v1_3.shp")

EHA_plant_level_hydropower_data.res_dam_source_id.dropna().duplicated().sum()

EHA_grand_ids = EHA_plant_level_hydropower_data.res_dam_source_id.dropna()

# Here two plants can have the same grand id. Therefore we need to take care of duplicates.
# Here isin function with work because we are making first time entries in EHA plant level data. Therefore we do not need to worry about duplicates
# However if we are updating the data then we need to take care of duplicates
filtered_grand_data = grand_data[grand_data.GRAND_ID.isin(EHA_grand_ids)]
filtered_grand_data = filtered_grand_data[['GRAND_ID', 'RES_NAME', 'DAM_NAME', 'RIVER', 'DAM_HGT_M', 'AREA_SKM', 'CAP_MCM', "DEPTH_M"]]
filtered_grand_data["res_vol_km3"] = filtered_grand_data["CAP_MCM"]*0.001 # Convert volume from Mm3 to km3
filtered_grand_data.drop(columns=["CAP_MCM"], inplace=True)
filtered_grand_data.rename(columns={"RES_NAME": "res_name", "DAM_NAME": "dam_name", "RIVER": "river", "DAM_HGT_M": "dam_height_m", "AREA_SKM": "res_area_km2", "DEPTH_M" : "res_avg_depth_m"}, inplace=True)
filtered_grand_data["res_attr_id"] = filtered_grand_data.GRAND_ID
filtered_grand_data["res_attr_source"] = "GranD"
filtered_grand_data["dam_height_source"] = "GranD"

EHA_plant_level_hydropower_data = pd.merge(EHA_plant_level_hydropower_data, filtered_grand_data, left_on='res_dam_source_id', right_on='GRAND_ID', how='left')
EHA_plant_level_hydropower_data.drop(columns=["GRAND_ID"], inplace=True)




/home/shah0012/.conda/envs/python_env/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
# RePP data
Africa_hydropower_data = pd.read_excel("/home/shah0012/GloHydroRes/Input_data/Rebecca_peters_Africa_data/RePP_Petersetal.xlsx", sheet_name= "S3 HPPD")
Africa_hydropower_data = gpd.GeoDataFrame(Africa_hydropower_data, geometry=gpd.points_from_xy(Africa_hydropower_data.lon, Africa_hydropower_data.lat), crs="EPSG:4326")
Africa_hydropower_data = Africa_hydropower_data[Africa_hydropower_data.stat_inf == "E"] # To get only the existing hydropower plants
Africa_hydropower_data = Africa_hydropower_data[["HPPD_ID", "lon", "lat", "country", "HPP_name", "type", "main_river", "g_hgt_m", "first_o_start", "g_cap_mw"]]
Africa_hydropower_data.rename(columns={"HPPD_ID": "plant_source_id", "HPP_name": "name", "g_hgt_m": "dam_height_m", 
                                       "first_o_start": "year", "main_river" : "river", "type" : "plant_type",
                                       "g_cap_mw" : "capacity_mw", "lon" : "plant_lon", "lat" : "plant_lat"}, inplace=True)
Africa_hydropower_data["dam_height_source"] = np.where(pd.isna(Africa_hydropower_data.dam_height_m), np.nan, "RePP")
Africa_hydropower_data["plant_source"] = "RePP"

def change_type(row):
    if row['plant_type'] == 'Reservoir':
        return 'STO'
    elif row['plant_type'] =='RoR':
        return 'ROR'
    elif row['plant_type'] == "Pumped Storage": 
        return "PS"
    elif row['plant_type'] == "RoR ":
        return "ROR"
    
Africa_hydropower_data["plant_type"] = Africa_hydropower_data.apply(change_type, axis=1)
Africa_hydropower_data["plant_type_source"] = "RePP"



In [8]:
columns = ["country", "name", "capacity_mw", "plant_lat", "plant_lon", "plant_type", "plant_type_source", "year",  
           "plant_source", "plant_source_id", "dam_name", "dam_height_m", "dam_height_source", "res_name", "res_dam_source", "res_dam_source_id",
             "man_dam_lat", "man_dam_lon", "river", "head_m", "head_source", "res_avg_depth_m", "res_area_km2", "res_vol_km3",
             "res_attr_source", "res_attr_id", "hydrolakes_id", "final_comments"] 

glohydrores =  pd.DataFrame(columns=columns)
glohydrores = pd.concat([glohydrores, WRI_hydro, JRC_hydropower_data, EHA_plant_level_hydropower_data, Africa_hydropower_data], axis=0)
glohydrores = gpd.GeoDataFrame(glohydrores, geometry=gpd.points_from_xy(glohydrores.plant_lon, glohydrores.plant_lat), crs="EPSG:4326")
glohydrores.set_index(np.arange(glohydrores.shape[0]), inplace=True)

In [9]:
#Each source of hydropower data has a unique plant_source_id. However, the unique IDs between the two datasets do not match, so we can use the plant_source_id for further analysis.
#Note: If two sources have similar IDs, such as those that begin number of sequence, this approach may not be suitable.
glohydrores.plant_source_id.duplicated().sum()

0

In [10]:
# DEM data
DEM_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/DEM", "tif", "hyd_glo_dem_15s.tif")

In [11]:
def get_nearest_values(row, other_gdf,  dam_lat_column = "LAT_DD", dam_lon_column = "LONG_DD",   value_column = 'geometry', unique_id = 'GRAND_ID'):
    """ Get nearest 3 dams for which distance between the dam and the hydro power plant is less than 10 km
    
    : param row: pd.Series: row of the GeoDataFrame containing the hydro power plant information
    : param other_gdf: gpd.GeoDataFrame: GeoDataFrame of the other dams
    : param dam_lat_column: str: column name of the latitude in dam dataset
    : param dam_lon_column: str: column name of the longitude in dam dataset
    : param value_column: str: column name of the geometry in the dam dataset
    : param unique_id: str: column name of the unique id in the dam dataset.
    : return nearest_id_list: list: list of the nearest dam ids (unique ids)
    
    """
    # Create an union of the other GeoDataFrame's geometeries
    nearest_id_list = []
    other_points = other_gdf[value_column].unary_union
    i = -1
    while i < 3: 
        
        nearest_geoms = nearest_points(row.geometry, other_points)
        idx = other_gdf[other_gdf[value_column] == nearest_geoms[1]].index
        idx = idx[0]
        sel_lat_val = row.plant_lat
        sel_lon_val = row.plant_lon
        nearest_ids = other_gdf.at[idx, unique_id]
        dam_lat =  other_gdf[dam_lat_column].to_numpy()[other_gdf[unique_id].to_numpy() == nearest_ids].item()
        dam_lon = other_gdf[dam_lon_column].to_numpy()[other_gdf[unique_id].to_numpy() == nearest_ids].item()        
        p1 = (dam_lat, dam_lon)
        p2 = (sel_lat_val, sel_lon_val)
        distance = geodesic(p1, p2).km

        if distance > 10:
            break
        else:
             nearest_id_list.append(nearest_ids)
             i += 1
        other_gdf = other_gdf.query(f'{unique_id} != {nearest_ids}')
        other_points = other_gdf[value_column].unary_union

    return nearest_id_list


##### Combine GloHydroRes with GRanD Data

In [12]:
def grand_manual_hydropower_reservoir_identification_func(dam_dataset : gpd.GeoDataFrame , hydropower_data, dem_data : xr.Dataset , unique_id  = 'GRAND_ID' ) -> pd.DataFrame:
    
    """ Identify the reservoir for the hydro power plants in the glohydrores dataset using the GranD dataset. Final dataframe will contain the information about plant id and 
        the identified reservoir id

    : param dam_dataset: gpd.GeoDataFrame: GeoDataFrame of the GranD dataset
    : param hydropower_data: pd.DataFrame: DataFrame of the hydro power plants
    : param dem_data: xr.Dataset: xarray Dataset of the DEM data
    : param unique_id: str: column name of the unique id in the dam dataset.
    : return df: pd.DataFrame: DataFrame of the identified reservoirs
    

    """

    #grand_res_id = dam_dataset[unique_id].to_list()
    plant_id_list = []
    matched_grand_id_list = []

    unmatched_hydropower_data = hydropower_data[hydropower_data['res_dam_source_id'].isna()]
    

    n = -1       
    for row in unmatched_hydropower_data.itertuples(index=False):
            n += 1
            print(n)
            plant_name = row.name
            plant_id = row.plant_source_id
            plant_lat = row.plant_lat
            plant_lon = row.plant_lon
            
            # Get hydropower plant location elevation from DEM data
            clipped_data = dem_data.sel(
                        y = plant_lat,
                        x = plant_lon,
                        method = "nearest")
        
            plant_dem = float(clipped_data.values) 

            # Get three nearest dams from the GranD dataset (distance between the dam and the hydro power plant is less than 10 km)                
            nearest_grand_ids = get_nearest_values(row, dam_dataset)

            ## BELOW TO CHECK IF NONE OF THE RESERVIOR IS LOCATED CLOSE TO 10KM

            if nearest_grand_ids:
                for id in nearest_grand_ids:
                    sel_dam_column = dam_dataset.query(f'{unique_id} == @id')
                    dam_name = str(sel_dam_column.DAM_NAME.iloc[0])
                    alt_name = str(sel_dam_column.ALT_NAME.iloc[0])
                    res_name = str(sel_dam_column.RES_NAME.iloc[0])
                    dam_lat = sel_dam_column.LAT_DD.iloc[0]
                    dam_lon = sel_dam_column.LONG_DD.iloc[0]
                    clipped_data = dem_data.sel(
                                y = dam_lat,
                                x = dam_lon, method = "nearest")
                    
                    # Get dam location elevation from DEM data
                    dam_dem = float(clipped_data.values)
                
                    # As grand provides detailed information about the resevoir, we are using dam name, alternative name and reservoir name to match the power plant with the dam
                    # Further if name does not match, we are using the elevation of the power plant and the dam to match the power plant with the dam
                    if (bool(set(plant_name.lower().split()).intersection(dam_name.lower().split())) or 
                        bool(set(plant_name.lower().split()).intersection(alt_name.lower().split())) or 
                        bool(set(plant_name.lower().split()).intersection(res_name.lower().split())) or 
                        plant_dem <= dam_dem):
                        matched_grand_id_list.append(id)
                        plant_id_list.append(plant_id)
                        break   
    df = pd.DataFrame({
        "plant_source_id" : plant_id_list,
        "res_dam_source_id" : matched_grand_id_list})
    return df


In [13]:
plant_id_matched_grand_id = grand_manual_hydropower_reservoir_identification_func(dam_dataset=grand_data, hydropower_data=glohydrores, dem_data=DEM_data)

0
1
2
3
4
5
6
7
8
9
10
11
12


KeyboardInterrupt: 

In [227]:
# First insert update data in the plant id and matched grand id dataframe
# Then update the glohydrores dataframe with updated dataframe

plant_id_matched_grand_id["dam_name"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.DAM_NAME)))
plant_id_matched_grand_id["dam_height_m"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.DAM_HGT_M)))
plant_id_matched_grand_id["dam_height_source"] = "GranD"
plant_id_matched_grand_id["res_name"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.RES_NAME)))
plant_id_matched_grand_id["res_dam_source"] = "GranD"
plant_id_matched_grand_id["river"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.RIVER)))
plant_id_matched_grand_id["res_avg_depth_m"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.DEPTH_M)))
plant_id_matched_grand_id["res_area_km2"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.AREA_SKM)))
plant_id_matched_grand_id["res_vol_km3"] = plant_id_matched_grand_id.res_dam_source_id.map(dict(zip(grand_data.GRAND_ID, grand_data.CAP_MCM)))
plant_id_matched_grand_id["res_vol_km3"] = plant_id_matched_grand_id["res_vol_km3"]*0.001 # Convert volume from Mm3 to km3
plant_id_matched_grand_id["res_attr_source"] = "GranD"
plant_id_matched_grand_id["res_attr_id"] = plant_id_matched_grand_id.res_dam_source_id


In [228]:

# First insert the matched grand id in the glohydrores dataset res_dam_id column
glohydrores["dam_name"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.dam_name)))
glohydrores["dam_height_m"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.dam_height_m)))
glohydrores["dam_height_source"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.dam_height_source)))
glohydrores["res_name"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_name)))
glohydrores["res_dam_source"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_dam_source)))
glohydrores["res_dam_source_id"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_dam_source_id)))
glohydrores["river"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.river)))
glohydrores["res_avg_depth_m"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_avg_depth_m)))
glohydrores["res_area_km2"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_area_km2)))
glohydrores["res_vol_km3"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_vol_km3)))
glohydrores["res_attr_source"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_attr_source)))
glohydrores["res_attr_id"] =  glohydrores.plant_source_id.map(dict(zip(plant_id_matched_grand_id.plant_source_id, plant_id_matched_grand_id.res_attr_id)))

/home/shah0012/.conda/envs/python_env/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


##### Combine GloHydroRes with GeoDAR data

In [14]:
#GeoDAR data
geodar_dam_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/GeoDAR_v10_v11/", "shp", "GeoDAR_v11_dams.shp")
geodar_dam_data = geodar_dam_data.to_crs('epsg:4326')

gdat_res_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/GDAT/GDAT_data_v1/data/", "shp", "GDAT_v1_catchments.shp")
gdat_res_data = gdat_res_data.to_crs('epsg:4326')

In [232]:
def get_gdat_data_func(n, dam_data):
    """ Get the GDAT data attributes for matched GeoDAR/HydroLakes reservoirs. GDAT reservoir is considered as matched 
    if the GeoDAR/HydroLakes dam is located within the GDAT reservoir  

    : param n: int: index of selected GeoDAR dam
    : param dam_data: gpd.GeoDataFrame: dam dataset of GeoDAR
    : return dam_name: str: name of the dam
    : return res_name: str: name of the reservoir
    : return river_name: str: name of the river
    : return dam_hgt: float: height of the dam
    : return dam_hgt_source: str: source of the dam height

    """

    # Here w
    if gdat_res_data[gdat_res_data.geometry.contains(dam_data.geometry[n])].empty:
        dam_name = None
        dam_height_m = None
        dam_height_source = None
        res_name = None
        river = None
        res_avg_depth_m = None
        res_area_km2 = None
        res_vol_km3 = None
        res_attr_source = None
        res_attr_id = None
    else: # Only return if there is only one matching
        sel_gdat_data = gdat_res_data[gdat_res_data.geometry.contains(dam_data.geometry[n])]
        if sel_gdat_data.shape[0] == 1: ## THIS IF ONLY ONE MATCHING  
            dam_name = sel_gdat_data.Dam_Name.iloc[0]
            dam_height_m = sel_gdat_data.Height.iloc[0]
            dam_height_source = "GDAT"
            res_name = sel_gdat_data.Reservoir.iloc[0]
            river = sel_gdat_data.River.iloc[0]
            res_avg_depth_m = sel_gdat_data.Avg_Depth.iloc[0]
            res_area_km2 = sel_gdat_data.Area_Con.iloc[0]
            res_vol_km3 = sel_gdat_data.Volume_Con.iloc[0]*0.001
            res_attr_source = "GDAT"
            res_attr_id = sel_gdat_data.Feature_ID.iloc[0]


        else: ## THIS IF MULTIPLE MATCHING
            dam_name = None
            dam_height_m = None
            dam_height_source = None
            res_name = None
            river = None
            res_avg_depth_m = None
            res_area_km2 = None
            res_vol_km3 = None
            res_attr_source = None
            res_attr_id = None

    return dam_name, dam_height_m, dam_height_source,  res_name, river, res_avg_depth_m, res_area_km2, res_vol_km3, res_attr_source, res_attr_id

In [233]:
def geodar_manual_hydropower_reservoir_identification_func(dam_dataset : gpd.GeoDataFrame, hydropower_data, dem_data : xr.Dataset, unique_id = 'id_v11') -> pd.DataFrame:
    """ Identify the reservoir for the hydro power plants in the glohydrores dataset using the GeoDAR/HydroLakes dataset. Final dataframe will contain the information about plant id, 
        the identified reservoir id, reservoir and dam attributes from the GDAT dataset


    : param dam_dataset: gpd.GeoDataFrame: GeoDataFrame of the GeoDAR/HydroLakes dataset
    : param hydropower_data: pd.DataFrame: DataFrame of the hydro power plants
    : param dem_data: xr.Dataset: xarray Dataset of the DEM data
    : param unique_id: str: column name of the unique id in the dam dataset.
    : return df: pd.DataFrame: DataFrame of the identified reservoirs

    """

    plant_id_list = []
    matched_id_list = []
    dam_name_list = []
    dam_height_list = []
    dam_height_source_list = []
    res_name_list = []
    river_list = []
    res_avg_depth_list = []
    res_area_list = []
    res_vol_list = []
    res_attr_source_list = []
    res_attr_id_list = []
    
    unmatched_hydropower_data = hydropower_data[hydropower_data['res_dam_source_id'].isna()]


    n = -1
    for row in unmatched_hydropower_data.itertuples(index=False):
        n += 1
        print(n)
        plant_id = row.plant_source_id
        sel_lat_val = row.plant_lat
        sel_lon_val = row.plant_lon
        clipped_data = dem_data.sel(
                        y = sel_lat_val,
                        x = sel_lon_val,
                        method = "nearest")
        
        plant_dem = float(clipped_data.values) 
        nearest_geodar_ids = get_nearest_values(row = row, other_gdf = dam_dataset,  dam_lat_column = "lat", dam_lon_column= "lon", unique_id="id_v11")

        if nearest_geodar_ids:
             for dam_id in nearest_geodar_ids:
                    sel_dam_column = dam_dataset.query(f'{unique_id} == @dam_id')
                    idx = sel_dam_column.index[0]
                    print(sel_dam_column.geometry)
                    dam_lat = sel_dam_column.lat.iloc[0]
                    dam_lon = sel_dam_column.lon.iloc[0]      
                    clipped_data = dem_data.sel(
                                y = dam_lat,
                                x = dam_lon, method = "nearest")
                    dam_dem = float(clipped_data.values)
                    if plant_dem <= dam_dem:
                         dam_name, dam_height_m, dam_height_source,  res_name, river, res_avg_depth_m, res_area_km2, res_vol_km3, res_attr_source, res_attr_id = get_gdat_data_func(idx, dam_data=geodar_dam_data)
                         plant_id_list.append(plant_id)
                         matched_id_list.append(dam_id)
                         dam_name_list.append(dam_name)
                         dam_height_list.append(dam_height_m)
                         dam_height_source_list.append(dam_height_source)
                         res_name_list.append(res_name)
                         river_list.append(river)
                         res_avg_depth_list.append(res_avg_depth_m)
                         res_area_list.append(res_area_km2)
                         res_vol_list.append(res_vol_km3)
                         res_attr_source_list.append(res_attr_source)
                         res_attr_id_list.append(res_attr_id)
                         break

     
    geodar_df = pd.DataFrame({
        "plant_source_id" : plant_id_list,
        "res_dam_source_id" : matched_id_list,
        "dam_name" : dam_name_list,
        "dam_height_m" : dam_height_list,
        "dam_height_source" : dam_height_source_list,
        "res_name" : res_name_list,
        "river" : river_list,
        "res_avg_depth_m" : res_avg_depth_list,
        "res_area_km2" : res_area_list,
        "res_vol_km3" : res_vol_list,
        "res_attr_source" : res_attr_source_list,
        "res_attr_id" : res_attr_id_list})
    
    return geodar_df


In [234]:
plant_id_matched_geodar_id = geodar_manual_hydropower_reservoir_identification_func(dam_dataset=geodar_dam_data, hydropower_data=glohydrores, dem_data=DEM_data)

0
1
2
3
127    POINT (19.86140 41.30800)
Name: geometry, dtype: geometry
82    POINT (19.90831 41.40153)
Name: geometry, dtype: geometry
4
112    POINT (19.89809 41.68145)
Name: geometry, dtype: geometry
28    POINT (19.83266 41.69192)
Name: geometry, dtype: geometry
5
23281    POINT (19.62749 42.02010)
Name: geometry, dtype: geometry
6
185    POINT (13.73193 -12.47061)
Name: geometry, dtype: geometry
7
8
9
10
227    POINT (-69.06667 -30.18333)
Name: geometry, dtype: geometry
11
12
13
14
15
225    POINT (-65.25145 -24.45026)
Name: geometry, dtype: geometry
16
17
220    POINT (-64.36183 -27.65094)
Name: geometry, dtype: geometry


In [235]:
plant_id_matched_geodar_id["res_dam_source"] = "GeoDAR"

In [236]:
# Update the glohydrores dataframe with the matched GeoDAR data

row_indices = glohydrores.loc[glohydrores.plant_source_id.isin(plant_id_matched_geodar_id.plant_source_id)].index
glohydrores.loc[row_indices, ["dam_name", "dam_height_m", "dam_height_source", "res_name", "river", "res_avg_depth_m", "res_area_km2", "res_vol_km3", "res_attr_source", "res_attr_id", "res_dam_source_id", "res_dam_source"]] = plant_id_matched_geodar_id[["dam_name", "dam_height_m", "dam_height_source", "res_name", "river", "res_avg_depth_m", "res_area_km2", "res_vol_km3", "res_attr_source", "res_attr_id", "res_dam_source_id", "res_dam_source"]].values



##### Combine GloHydroRes with HydroLakes data

In [17]:
#HydroLakes data
hydrolakes_point_data = load_input_data("/home/shah0012/GloHydroRes/Input_data/HydroLakes/HydroLAKES_points_v10_shp/", "shp", "HydroLAKES_points_v10.shp")

In [250]:
def get_gdat_data_hydrolakes_func(dam_data):
    """ Get the GDAT data attributes for matched HydroLakes reservoirs. GDAT reservoir is considered as matched 
    if the HydroLakes dam is located within the GDAT reservoir  

    : param dam_data: gpd.GeoDataFrame: dam dataset of HydroLakes
    : return dam_name: str: name of the dam
    : return res_name: str: name of the reservoir
    : return river_name: str: name of the river
    : return dam_hgt: float: height of the dam
    : return dam_hgt_source: str: source of the dam height

    """

    # Here w
    if gdat_res_data[gdat_res_data.geometry.contains(dam_data.geometry.iloc[0])].empty:
        dam_name = None
        dam_height_m = None
        dam_height_source = None
        res_name = None
        river = None
    else: # Only return if there is only one matching
        sel_gdat_data = gdat_res_data[gdat_res_data.geometry.contains(dam_data.geometry.iloc[0])]
        if sel_gdat_data.shape[0] == 1: ## THIS IF ONLY ONE MATCHING  
            dam_name = sel_gdat_data.Dam_Name.iloc[0]
            dam_height_m = sel_gdat_data.Height.iloc[0]
            dam_height_source = "GDAT"
            res_name = sel_gdat_data.Reservoir.iloc[0]
            river = sel_gdat_data.River.iloc[0]

        else: ## THIS IF MULTIPLE MATCHING
            dam_name = None
            dam_height_m = None
            dam_height_source = None
            res_name = None
            river = None
            
    return dam_name, dam_height_m, dam_height_source, res_name, river





def hydrolakes_manual_hydropower_reservoir_identification_func(dam_dataset : gpd.GeoDataFrame, hydropower_data, dem_data : xr.Dataset, unique_id = 'Hylak_id') -> pd.DataFrame:
    """ Identify the reservoir for the hydro power plants in the glohydrores dataset using the HydroLakes dataset. Final dataframe will contain the information about plant id and 
        the identified reservoir id


    : param dam_dataset: gpd.GeoDataFrame: GeoDataFrame of the HydroLakes dataset
    : param hydropower_data: pd.DataFrame: DataFrame of the hydro power plants
    : param dem_data: xr.Dataset: xarray Dataset of the DEM data
    : param unique_id: str: column name of the unique id in the dam dataset.
    : return df: pd.DataFrame: DataFrame of the identified reservoirs

    """

    plant_id_list = []
    matched_hydlak_id_list = []
    dam_name_list = []
    dam_height_list = []
    dam_height_source_list = []
    res_name_list = []
    river_list = []
    res_avg_depth_list = []
    res_area_list = []
    res_vol_list = []
    res_attr_source_list = []
    res_attr_id_list = []
    
    unmatched_hydropower_data = hydropower_data[hydropower_data['res_dam_source_id'].isna()]


    n = -1
    for row in unmatched_hydropower_data.itertuples(index=False):
        n += 1
        print(n)
        plant_id = row.plant_source_id
        sel_lat_val = row.plant_lat
        sel_lon_val = row.plant_lon
        clipped_data = dem_data.sel(
                        y = sel_lat_val,
                        x = sel_lon_val,
                        method = "nearest")
        
        plant_dem = float(clipped_data.values) 
        nearest_hydrolakes_ids = get_nearest_values(row = row, other_gdf = dam_dataset,  dam_lat_column = "Pour_lat", dam_lon_column= "Pour_long", unique_id="Hylak_id")
        print(n)

        if nearest_hydrolakes_ids:
             for dam_id in nearest_hydrolakes_ids:
                    sel_dam_column = dam_dataset.query(f'{unique_id} == @dam_id')
                    dam_lat = sel_dam_column.Pour_lat.iloc[0]
                    dam_lon = sel_dam_column.Pour_long.iloc[0]
                    clipped_data = dem_data.sel(
                                y = dam_lat,
                                x = dam_lon, method = "nearest")
                    dam_dem = float(clipped_data.values)

                    if plant_dem <= dam_dem:
                        res_avg_depth_m = sel_dam_column.Depth_avg.iloc[0]
                        res_area_km2 = sel_dam_column.Lake_area.iloc[0]
                        res_vol_km3 = sel_dam_column.Vol_total.iloc[0]
                        res_attr_source = "HydroLakes"
                        res_attr_id = sel_dam_column.Hylak_id.iloc[0]
                        
                        plant_id_list.append(plant_id)
                        matched_hydlak_id_list.append(dam_id)
                        res_avg_depth_list.append(res_avg_depth_m)
                        res_area_list.append(res_area_km2)
                        res_vol_list.append(res_vol_km3)
                        res_attr_source_list.append(res_attr_source)
                        res_attr_id_list.append(res_attr_id)
                         
                        dam_name, dam_height_m, dam_height_source, res_name, river = get_gdat_data_hydrolakes_func(dam_data=sel_dam_column)
                        dam_name_list.append(dam_name)
                        dam_height_list.append(dam_height_m)
                        dam_height_source_list.append(dam_height_source)
                        res_name_list.append(res_name)
                        river_list.append(river)
                        break
     
    hydroalakes_df = pd.DataFrame({
        "plant_source_id" : plant_id_list,
        "res_dam_source_id" : matched_hydlak_id_list,
        "dam_name" : dam_name_list,
        "dam_height_m" : dam_height_list,
        "dam_height_source" : dam_height_source_list,
        "res_name" : res_name_list,
        "river" : river_list,
        "res_avg_depth_m" : res_avg_depth_list,
        "res_area_km2" : res_area_list,
        "res_vol_km3" : res_vol_list,
        "res_attr_source" : res_attr_source_list,
        "res_attr_id" : res_attr_id_list})
    
    return hydroalakes_df


In [251]:
plant_id_matched_hydrlakes_id = hydrolakes_manual_hydropower_reservoir_identification_func(dam_dataset=hydrolakes_point_data, hydropower_data=glohydrores, dem_data=DEM_data)

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12


In [256]:
plant_id_matched_hydrlakes_id["res_dam_source"] = "HydroLakes"

In [257]:
# Update the glohydrores dataframe with the matched HydroLakes data
row_indices = glohydrores.loc[glohydrores.plant_source_id.isin(plant_id_matched_hydrlakes_id.plant_source_id)].index
glohydrores.loc[row_indices, ["dam_name", "dam_height_m", "dam_height_source", "res_name", "river", "res_avg_depth_m", "res_area_km2", "res_vol_km3", "res_attr_source", "res_attr_id", "res_dam_source_id", "res_dam_source"]] = plant_id_matched_hydrlakes_id[["dam_name", "dam_height_m", "dam_height_source", "res_name", "river", "res_avg_depth_m", "res_area_km2", "res_vol_km3", "res_attr_source", "res_attr_id", "res_dam_source_id", "res_dam_source"]].values

In [258]:
glohydrores.to_excel("/home/shah0012/GloHydroRes_data/output_data/glohydrores_v1.xlsx", sheet_name = "data")

/home/shah0012/.conda/envs/python_env/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
